In [1]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
from scipy.optimize import curve_fit

In [3]:
data = load('Temperature_DC05V_RF64V.npz')
time = data['time']
Temp = data['temperature']
i = argmin(abs(time-0.002))

In [4]:
figure(1); clf();
semilogy(time[i:], Temp[i:])
grid()

In [5]:
x0 = (time[i:]-time[i])*1e3
y0 = log10(Temp[i:])
imax = argmax(diff(y0))

In [9]:
def func(x0, A, B, C, D, t0,E):
    return A*(x0-t0) / (D+C*(x0-t0)**2)**(1/E) + B

t0 = x0[imax]
A  = 1
B  = 0
C  = 1
D  = 1
E  = 2.5
G  = 0.5
p0 = array([A, B, C, D, t0,E])
popt, pcov = curve_fit(func, x0, y0, p0)
y1 = func(x0,popt[0], popt[1], popt[2], popt[3], popt[4], popt[5])
Temp_fit = 10**(y1)

<ipython-input-9-c49b7730026a>:2: RuntimeWarning: invalid value encountered in power
  return A*(x0-t0) / (D+C*(x0-t0)**2)**(1/E) + B


In [10]:
figure(2); clf();
subplot(311)
plot(x0,y0)
plot(x0,y1,label='x/(1+x^2)')
grid()

subplot(212)
plot(x0,Temp[i:])
plot(x0,Temp_fit,label='x/(1+x^2)')
grid()


In [16]:
dt = (x0[1] - x0[0])*1e-3 # Because I had put x0 in ms
heating_rate = diff(Temp_fit) / dt

figure(2); clf();
semilogy(x0[:-1],heating_rate,label='x/(1+x^2)')
xlabel('time[ms]')
ylabel('Heating Rate [K/s]')
grid()


In [20]:
figure(2); clf();
subplot(211)
semilogy(x0,Temp[i:], label='Data')
semilogy(x0,Temp_fit , label='Fit')
grid()
xlabel('time[ms]')
ylabel('Temperature[K]')

subplot(212)
semilogy(x0[:-1],heating_rate,label='x/(1+x^2)')
grid()
xlabel('time[ms]')
ylabel('Heating Rate [K/s]')


Text(0, 0.5, 'Heating Rate [K/s]')

In [21]:
savefig('Heating_Rate_from_Fit.png')